# Police Force Responses by Crime

In [ ]:
#Dependencies
import matplotlib
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd

In [ ]:
#Import csv file
police_data = pd.read_csv("Resources/Police_Use_of_Force.csv")
police_data.head()

In [ ]:
police_data.columns

In [ ]:
police_data_df = police_data.loc[:, ["PoliceUseOfForceID", "CaseNumber", "ResponseDate", "Problem", "ForceType",
                                    "ForceTypeAction", "EventAge", "TypeOfResistance", "Precinct", "Neighborhood"]]
police_data_df

In [ ]:
police_data_df.dtypes

In [ ]:
police_data_df = police_data_df.dropna(how='any')
police_data_df

In [ ]:
clean_df = police_data_df.drop_duplicates(subset=['CaseNumber'])
clean_df

# 1. What incidents result the most in police use of force? (Sam)

# 2. What type of force is used the most? (Ainash)

# 3. Which is the most popular type of resistance? (Humera)

# Most popular type of Resistance

In [ ]:
popular_data = clean_df["TypeOfResistance"].describe(include= "all")
popular_data

In [ ]:
resistance_data = clean_df.groupby("TypeOfResistance")
total_count = resistance_data.nunique()["CaseNumber"]

resistance = pd.DataFrame({"Total Count": total_count})
resistance_case_no = resistance.sort_values("Total Count", ascending=False)

resistance_case_no.index.name = "TypeOfResistance"
resistance_case_no.head()

In [ ]:
resistance_case_no.head(6).plot(kind="bar",edgecolor="black", figsize=(8, 8),facecolor="blue", fontsize=10)
plt.title("Type of Resistance Resulting in Use of Police Force in Minneapolis")
plt.ylabel("Number of Cases", fontsize=12)
plt.xlabel("Type of Resistance", fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
#Pie chart
colors = ['gold', 'red', 'pink', 'blue', 'lightskyblue']
explode = (0.1, 0, 0, 0, 0)
resistance_case_no.head(5).plot(kind="pie",explode=explode, colors=colors, autopct= "%1.1f%%", subplots="False", shadow=True, startangle=150 )
plt.title('Top 5 - Type of Resistance Resulting in Use of Police Force in Minneapolis', fontsize=10)
plt.axis('equal')
plt.tight_layout()
plt.show()

# Force Type of Action used by Police Force in Minneapolis

In [ ]:
popular_data = clean_df["ForceTypeAction"].describe(include= "all")
popular_data

In [ ]:
forcetype_data = clean_df.groupby("ForceTypeAction")
total_count = forcetype_data.nunique()["CaseNumber"]

forcetype = pd.DataFrame({"Total Count": total_count})
forcetype_case_no = forcetype.sort_values("Total Count", ascending=False)

forcetype_case_no.index.name = "ForceTypeAction"
forcetype_case_no.head()

In [ ]:
forcetype_case_no.head(5)
forcetype_case_no.head(10).plot(kind="barh",edgecolor="black", facecolor="red", fontsize=12, align="center", alpha=1.0)
plt.title("ForceTypeAction Resulting in Use of Police Force in Minneapolis")
plt.ylabel("Number of Cases", fontsize=12)
plt.xlabel("Force Type Action", fontsize=12)
plt.show()

In [ ]:
clean_df

# Groupby Forcetype and Type of Resistance

In [ ]:
#Top 5 - Force Type and Force Type Action Vs Number of Cases
ft_fa = clean_df.groupby(['ForceType','ForceTypeAction'])
ft_fa.count().head(5)

In [ ]:
#Top 5 - Force Type and Force Type Action Vs Number of Cases
ft_fa= clean_df.groupby(['ForceType','ForceTypeAction'])
total_count = ft_fa.nunique()["CaseNumber"]
forcetype_forceaction = pd.DataFrame({"Total Count": total_count})
forcetype_forceaction = forcetype_forceaction.sort_values("Total Count", ascending=False)
#forcetype_forceaction.index.name = "ForceTypeAction"
forcetype_forceaction.head()

In [ ]:
#Top 5 - Force Type and Force Type Action Vs Number of Cases
forcetype_forceaction.head(8).plot(kind="barh",edgecolor="black", facecolor="purple", fontsize=12, align="center", alpha=1.0)
plt.title("Force Type and Force Type Action Vs Number of Cases")
plt.ylabel("Force Type Action_Force Type", fontsize=12)
plt.xlabel("Number of Cases", fontsize=12)
plt.show()

In [ ]:
#Top 5 - Force Type and Type of Resistance Vs Number of Cases
ft_tr = clean_df.groupby(['ForceType','TypeOfResistance'])
ft_tr.count().head(5)

In [ ]:
# Top 5 - Force Type and Type of Resistance Vs Number of Cases
ft_tr= clean_df.groupby(['ForceType','TypeOfResistance'])
total_count = ft_tr.nunique()["CaseNumber"]
forcetype_resistance = pd.DataFrame({"Total Count": total_count})
forcetype_resistance = forcetype_resistance.sort_values("Total Count", ascending=False)
forcetype_resistance.head()

In [ ]:
# Top 5 - Force Type and Type of Resistance Vs Number of Cases
forcetype_resistance.head(8).plot(kind="barh",edgecolor="black", facecolor="orange", fontsize=12, align="center", alpha=1.0)
plt.title("Force Type and Type of Resistance Vs Number of Cases")
plt.ylabel("Force Type_ Resistance", fontsize=12)
plt.xlabel("Number of Cases", fontsize=12)
plt.show()

In [ ]:
#problem and force type

In [ ]:
#Problem and Type of Resistance

In [ ]:
# Problem and force type

# 4. Which neighborhood in Minneapolis see’s the highest frequency of police force? (Mike Hills)

# 5. Is the use of police force in Minneapolis going up or down? (Mike F)

# 6. Perform a linear regression on use of force between 2015 and 2019 and make a prediction about use of force in 2020 for the city of Minneapolis.  Compare this prediction to actual use of force outcomes in 2020.